# Jupyter Notebook에서 Django 돌리기

In [1]:
import django
import os

SECRET_KEY = "apiserver"
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': ':memory:',
    }
}
ROOT_URLCONF = '__main__'

urls = []

os.environ['DJANGO_SETTINGS_MODULE'] = '__main__'

django.setup()

In [2]:
django.get_version()

'2.0.1'

# Form 클래스 사용하기

In [3]:
from django import forms
def odd_validator(value):
    if value % 2 == 0:
        raise forms.ValidationError('짝수라니!!')
        
class QuizForm(forms.Form):
    answer = forms.IntegerField(validators=[odd_validator])

In [4]:
data = {'answer': 10}

In [5]:
form = QuizForm(data)

In [6]:
# answer값이 짝수라서, 유효성 검사 실패
form.is_valid()

False

In [7]:
form.errors

{'answer': ['짝수라니!!']}

# Model 클래스 사용하기

In [8]:
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        app_label = 'api'  # 앱이 따로 없으므로, app label을 필히 지정해줘야합니다.
    
    def __str__(self):
        return self.title

jupyter notebook에서 django를 운영하기 때문에 migration을 하지 못하여, 모델 클래스 내역대로 Raw SQL로 DB 테이블을 생성합니다.

In [9]:
from django.db import connection

table_name = Post._meta.db_table

with connection.cursor() as cursor:
    cursor.execute('''
CREATE TABLE "{}"
    ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
    "title" varchar(100) NOT NULL,
    "content" text NOT NULL,
    "created_at" datetime NOT NULL,
    "updated_at" datetime NOT NULL);
'''.format(table_name))

In [10]:
with connection.cursor() as cursor:
    cursor.execute("select name from sqlite_master where type = 'table';")
    for row in cursor.fetchall():
        print(row)

('api_post',)
('sqlite_sequence',)


#### Post 모델을 통해 DB 데이터 추가 

In [11]:
Post.objects.create(
    title="가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯",
    content="소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다. 오늘 본사 차원에서 한국 지역 애플 스토어 지니어스를 포함한 리더 프로그램, 스토어 리더 인력 채용 공고를 냈다. 모두 이전까지는 공고를 낸 적이 없는 직책들이다. 애플코리아도 가로수길 애플 스토어 오픈을 공식화했다. 관계자에 따르면 “세계 경제의 중심지 중 하나이고, 통신 및 기술 분야의 리더이자 세계적으로 사랑 받는 한류 문화를 갖춘 대한민국에 첫 번째 애플 스토어를 오픈하게 되어 매우 기쁘다. 현재 서울에서 서비스, 교육 및 전 세계 애플 고객들에게 다양한 엔터테인먼트를 제공할 팀을 채용 중이다.”고 말했다. 가로수길에 들어설 한국의 첫 애플 스토어는 미국 샌프란시스코 유니언스퀘어에 처음 문을 연 '2세대 애플 스토어' 형태가 될 것으로 보인다. 체험 중심의 1세대와 달리 2세대 애플 스토어는 주변 환경과 사람 간의 교감을 우선으로 고려한 것이 특징이다. 2층 높이의 전면 유리막이 온도, 날씨에 의해 자동 개폐되고 애플 직원들이 대기하며 고객의 궁금증 해소를 돕는다. 전 세계 20여 개국 400여개가 운영되고 있는 오프라인 애플 스토어가 들어섬에 따라 1:1 전문 상담 창구인 '지니어스바' 서비스를 받을 수 있게 된다. 프리스비 같은 애플 프리미엄 리셀러의 미흡했던 고객지원 개선이 기대된다. 현재는 고장 정도에 상관없이 짧게는 1주일 심하면 몇 주를 서비스센터에 맡겨야 한다. 국내에선 불가능하던 아이폰 애플케어+를 가입할 수도 있다. 약 5만 원의 추가 비용을 지불하고 무료 보증 기간을 2년으로 연장, 소비자 과실로 발생하는 문제에 대해 좀 더 저렴한 가격으로 서비스를 받을 수 있다. 또 아이튠즈 스토어, 아이폰 찾기 같은 핵심 서비스 지원도 가능성이 있다. 아이폰 등 주요 신제품의 1차 출시국 포함도 기대된다. 한국 애플 스토어가 들어설 서울 강남구 신사동 가로수길 토지의 1층 면적은 약 159평이고, 2~3층 규모 건물이 지어질 것으로 보인다. 2015년 기준 전 세계 오프라인 애플 스토어가 운영 중인 나라는 미국(266개), 영국(39개), 캐나다(29개), 호주(21개), 중국(19개), 프랑스(19개), 일본(6개) 등이다."
    )

<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>

In [13]:
Post.objects.create(
    title="공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 ",
    content="국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.")


<Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >

In [15]:
Post.objects.create(
    title="규제 해커톤 또 무산 위기…카풀업계 속앓이 ",
    content="내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다. 카풀업계에선 어떤 결론도 나지 않아 속이 타는 상황이 이어지고 있다.22일 업계에 따르면 택시업계는 이날 오후 4차산업혁명위를 방문해 해커톤 불참을 알리고 불참에 대한 당위성을 설명했다. 위원회는 앞서 지난해 말 열린 1차 해커톤에 카풀 규제 관련 의제를 논의할 예정이었지만, 택시업계의 불참으로 불발됐다. 카풀과 택시업계의 갈등은 카풀 앱 '풀러스'가 시간선택제 운영을 도입한다고 밝히면서 촉발됐다. 시간선택제는 이용자의 실제 출퇴근 시간에 해당하는 시간과 요일을 지정해 주 5일만 이용하고, 변경은 월 1회로 제한하는 방식이다. 이전까지 카풀 앱은 출근 시간(오전 5시~11시)과 퇴근시간(오후 5시~익일 오전 2시)을 지정해 서비스를 운영하고 있었다.")

<Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >

In [16]:
Post.objects.all()

<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>

## ModelForm 사용하기

In [17]:
class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = '__all__'

In [18]:
form = PostForm({'title': 'hello'})

In [19]:
form.is_valid()

False

In [20]:
form.errors

{'content': ['This field is required.']}

In [21]:
for post in Post.objects.all():
    print(post.id, post.title, ':', len(post.content), '글자')

1 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯 : 1119 글자
2 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반  : 77 글자
3 규제 해커톤 또 무산 위기…카풀업계 속앓이  : 454 글자


# DjangoJSONEncoder

In [23]:
import json
from django.core.serializers.json import DjangoJSONEncoder

In [24]:
data = Post.objects.all()
data

<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>

TypeError: Object of type 'QuerySet' is not JSON serializable 예외처리

In [25]:
json.dumps(data, cls=DjangoJSONEncoder)

TypeError: <QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]> is not JSON serializable

DjangoJSONEncoder는 QuerySet의 serializer/deserializer 방법을 모르기 때문에 발생한 요류

1. **QuerySet**을 파이썬 표준 테이터타입의 값을 직접 변환

In [31]:
data = [
    {'id': post.id, 'title': post.title, 'content': post.content}
    for post in Post.objects.all()]

json.dumps(data, cls=DjangoJSONEncoder, ensure_ascii=False)

'[{"id": 1, "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯", "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다. 오늘 본사 차원에서 한국 지역 애플 스토어 지니어스를 포함한 리더 프로그램, 스토어 리더 인력 채용 공고를 냈다. 모두 이전까지는 공고를 낸 적이 없는 직책들이다. 애플코리아도 가로수길 애플 스토어 오픈을 공식화했다. 관계자에 따르면 “세계 경제의 중심지 중 하나이고, 통신 및 기술 분야의 리더이자 세계적으로 사랑 받는 한류 문화를 갖춘 대한민국에 첫 번째 애플 스토어를 오픈하게 되어 매우 기쁘다. 현재 서울에서 서비스, 교육 및 전 세계 애플 고객들에게 다양한 엔터테인먼트를 제공할 팀을 채용 중이다.”고 말했다. 가로수길에 들어설 한국의 첫 애플 스토어는 미국 샌프란시스코 유니언스퀘어에 처음 문을 연 \'2세대 애플 스토어\' 형태가 될 것으로 보인다. 체험 중심의 1세대와 달리 2세대 애플 스토어는 주변 환경과 사람 간의 교감을 우선으로 고려한 것이 특징이다. 2층 높이의 전면 유리막이 온도, 날씨에 의해 자동 개폐되고 애플 직원들이 대기하며 고객의 궁금증 해소를 돕는다. 전 세계 20여 개국 400여개가 운영되고 있는 오프라인 애플 스토어가 들어섬에 따라 1:1 전문 상담 창구인 \'지니어스바\' 서비스를 받을 수 있게 된다. 프리스비 같은 애플 프리미엄 리셀러의 미흡했던 고객지원 개선이 기대된다. 현재는 고장 정도에 상관없이 짧게는 1주일 심하면 몇 주를 서비스센터에 맡겨야 한다. 국내에선 불가능하던 아이폰 애플케어+를 가입할 수도 있다. 약 5만 원의 추가 비용을 지불하고 무료 보증 기간을 2년으로 연장, 소비자 과실로 발생하는 문제에 대해 좀 더 저렴한 가격으로 서비스를 받을 수 있다. 또 아이튠즈 스토어, 아이폰 찾기 같은 핵심 서비스 지원도 가능성이 있다. 아이폰 등 주요 신제품의 1차 출시국 포

**ensure_ascii=False** 는 한글 인코딩시 유니코드가 아닌 UTF-8로 인코딩합니다.
api서버에서는 상관없지만 UTF-8로 인코딩하여야 사람이 보기 편함

In [35]:
import json

mydata = ['안녕', '파파파파파파이썬']
json.dumps(mydata)

'["\\uc548\\ub155", "\\ud30c\\ud30c\\ud30c\\ud30c\\ud30c\\ud30c\\uc774\\uc36c"]'

In [36]:
json.dumps(mydata, ensure_ascii=False)

'["안녕", "파파파파파파이썬"]'

.2. json에게 serializer 방법을 알려준다.  
DjangoJSONEncoder가 직렬화 방법을 알고 있기 때문에 이를 확장한다.

    - QuerySet 타입 : tuple 타입으로 변환
    - Post 타입 : dict 타입으로 변환

In [37]:
from django.core.serializers.json import DjangoJSONEncoder
from django.db.models.query import QuerySet

# 커스텀 JSON Encoder를 정의
class MyJSONEncoder(DjangoJSONEncoder):
    def default(self, obj):
        if isinstance(obj, QuerySet):
            return tuple(obj)
        elif isinstance(obj, Post):
            return {'id': obj.id, 'title': obj.title, 'content': obj.content}
        return super().default(obj)

data = Post.objects.all()

# 직렬화할 때, 직렬화를 수행해줄 JSON Encoder를 지정해준다.
json.dumps(data, cls=MyJSONEncoder, ensure_ascii=False)

'[{"id": 1, "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯", "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다. 오늘 본사 차원에서 한국 지역 애플 스토어 지니어스를 포함한 리더 프로그램, 스토어 리더 인력 채용 공고를 냈다. 모두 이전까지는 공고를 낸 적이 없는 직책들이다. 애플코리아도 가로수길 애플 스토어 오픈을 공식화했다. 관계자에 따르면 “세계 경제의 중심지 중 하나이고, 통신 및 기술 분야의 리더이자 세계적으로 사랑 받는 한류 문화를 갖춘 대한민국에 첫 번째 애플 스토어를 오픈하게 되어 매우 기쁘다. 현재 서울에서 서비스, 교육 및 전 세계 애플 고객들에게 다양한 엔터테인먼트를 제공할 팀을 채용 중이다.”고 말했다. 가로수길에 들어설 한국의 첫 애플 스토어는 미국 샌프란시스코 유니언스퀘어에 처음 문을 연 \'2세대 애플 스토어\' 형태가 될 것으로 보인다. 체험 중심의 1세대와 달리 2세대 애플 스토어는 주변 환경과 사람 간의 교감을 우선으로 고려한 것이 특징이다. 2층 높이의 전면 유리막이 온도, 날씨에 의해 자동 개폐되고 애플 직원들이 대기하며 고객의 궁금증 해소를 돕는다. 전 세계 20여 개국 400여개가 운영되고 있는 오프라인 애플 스토어가 들어섬에 따라 1:1 전문 상담 창구인 \'지니어스바\' 서비스를 받을 수 있게 된다. 프리스비 같은 애플 프리미엄 리셀러의 미흡했던 고객지원 개선이 기대된다. 현재는 고장 정도에 상관없이 짧게는 1주일 심하면 몇 주를 서비스센터에 맡겨야 한다. 국내에선 불가능하던 아이폰 애플케어+를 가입할 수도 있다. 약 5만 원의 추가 비용을 지불하고 무료 보증 기간을 2년으로 연장, 소비자 과실로 발생하는 문제에 대해 좀 더 저렴한 가격으로 서비스를 받을 수 있다. 또 아이튠즈 스토어, 아이폰 찾기 같은 핵심 서비스 지원도 가능성이 있다. 아이폰 등 주요 신제품의 1차 출시국 포

## rest_framework.renderer.JSONRender의 직렬화 방식